# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

import warnings
warnings.filterwarnings("ignore")
# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df['Wind Speed'].sort_values()

385     0.00
332     0.00
533     0.00
439     0.00
7       0.00
       ...  
260    12.35
21     12.86
352    13.07
0      13.08
5      17.85
Name: Wind Speed, Length: 545, dtype: float64

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
# Configure the map plot
map_plot_humid = city_data_df.hvplot.points(
                    "Lng",
                    "Lat",
                    geo=True,
                    tiles="OSM",
                    frame_width=500,
                    frame_height=300,
                    size="Humidity",
                    scale=0.01,
                    color="City"
)

# Display the map
map_plot_humid

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city = city_data_df[(city_data_df['Max Temp'] > 120) & 
                          (city_data_df['Max Temp'] <200 ) &
                          (city_data_df['Humidity'] < 70) | 
                          (city_data_df['Wind Speed'] == 0)]

# Drop any rows with null values
ideal_city = ideal_city.dropna()

# Display sample data
ideal_city

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,fairbanks,64.8378,-147.7164,260.15,84,0,0.0,US,1678563000
247,247,alice springs,-23.7000,133.8833,288.90,41,0,0.0,AU,1678562779
332,332,nome,64.5011,-165.4064,264.19,79,100,0.0,US,1678562738
385,385,north bend,43.4065,-124.2243,285.51,90,40,0.0,US,1678563332
439,439,roros,62.5747,11.3842,257.06,85,77,0.0,NO,1678563354
533,533,college,64.8569,-147.8028,260.14,84,20,0.0,US,1678563395


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
7,fairbanks,US,64.8378,-147.7164,84,
247,alice springs,AU,-23.7000,133.8833,41,
332,nome,US,64.5011,-165.4064,79,
385,north bend,US,43.4065,-124.2243,90,
439,roros,NO,62.5747,11.3842,85,
533,college,US,64.8569,-147.8028,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, 'Lat']
    longitude = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fairbanks - nearest hotel: Westmark Fairbanks Hotel
alice springs - nearest hotel: Aurora Alice Springs
nome - nearest hotel: No hotel found
north bend - nearest hotel: The Mill Casino Hotel
roros - nearest hotel: Bergstadens Hotel
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
7,fairbanks,US,64.8378,-147.7164,84,Westmark Fairbanks Hotel
247,alice springs,AU,-23.7000,133.8833,41,Aurora Alice Springs
332,nome,US,64.5011,-165.4064,79,No hotel found
385,north bend,US,43.4065,-124.2243,90,The Mill Casino Hotel
439,roros,NO,62.5747,11.3842,85,Bergstadens Hotel
533,college,US,64.8569,-147.8028,84,BEST WESTERN PLUS Pioneer Park Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
                    "Lng",
                    "Lat",
                    geo=True,
                    tiles="OSM",
                    frame_width=500,
                    frame_height=300,
                    color="City",
                    hover_cols=['Hotel Name', 'Country'])

# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)